# LLM and RAG Evaluation

Sources: [1](https://docs.llamaindex.ai/en/stable/module_guides/evaluating/), [2](https://docs.llamaindex.ai/en/stable/examples/evaluation/faithfulness_eval/)  

LLMs are trained on enormous bodies of data but they aren’t trained on your data. Retrieval-Augmented Generation (RAG) solves this problem by adding your data to the data LLMs already have access to. You will see references to RAG frequently in this documentation.  
In RAG, your data is loaded and prepared for queries or “indexed”. User queries act on the index, which filters your data down to the most relevant context. This context and your query then go to the LLM along with a prompt, and the LLM provides a response.  
Even if what you’re building is a chatbot or an agent, you’ll want to know RAG techniques for getting data into your application.  

Evaluation and benchmarking are crucial concepts in LLM development. To improve the performance of an LLM app (RAG, agents), you must have a way to measure it.

LlamaIndex offers key modules to measure the quality of generated results. We also offer key modules to measure retrieval quality.

## 1.Response Evaluation:  
Does the response match the retrieved context? Does it also match the query? Does it match the reference answer or guidelines?

## 2. Retrieval Evaluation:  
Are the retrieved sources relevant to the query?

This section describes how the evaluation components within LlamaIndex work.

---  
## 1. Response Evaluation

Evaluation of generated results can be difficult, since unlike traditional machine learning the predicted result isn't a single number, and it can be hard to define quantitative metrics for this problem. LlamaIndex offers LLM-based evaluation modules to measure the quality of results. This uses a "gold" LLM (e.g. GPT-4) to decide whether the predicted answer is correct in a variety of ways. Note that many of these current evaluation modules do not require ground-truth labels. Evaluation can be done with some combination of the query, context, response, and combine these with LLM calls.

These evaluation modules are in the following forms:

+ #### Correctness: Whether the generated answer matches that of the reference answer given the query (requires labels).
+ #### Semantic Similarity Whether the predicted answer is semantically similar to the reference answer (requires labels).
+ #### Faithfulness: Evaluates if the answer is faithful to the retrieved contexts (in other words, whether if there's hallucination).
+ #### Context Relevancy: Whether retrieved context is relevant to the query.
+ #### Answer Relevancy: Whether the generated answer is relevant to the query.
+ #### Guideline Adherence: Whether the predicted answer adheres to specific guidelines.

+ #### Question Generation: In addition to evaluating queries, LlamaIndex can also use your data to generate questions to evaluate on. This means that you can automatically generate questions, and then run an evaluation pipeline to test if the LLM can actually answer questions accurately using your data.

---
## 2. Retrieval Evaluation (TBD)

We also provide modules to help evaluate retrieval independently.

The concept of retrieval evaluation is not new; given a dataset of questions and ground-truth rankings, we can evaluate retrievers using ranking metrics like mean-reciprocal rank (MRR), hit-rate, precision, and more.

The core retrieval evaluation steps revolve around the following:

+ #### Dataset generation: Given an unstructured text corpus, synthetically generate (question, context) pairs.  
+ #### Retrieval Evaluation: Given a retriever and a set of questions, evaluate retrieved results using ranking metrics.  
--- 

#### Installing Packages

In [104]:
import warnings
warnings.filterwarnings("ignore")

In [105]:
!pip install -qU openai
!pip install -qU llama-index
!pip install -qU llama-index-experimental
!pip install -qU llama-index-llms-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.12.10 requires llama-index-core<0.13.0,>=0.12.10, but you have llama-index-core 0.11.23 which is incompatible.
llama-index 0.12.10 requires llama-index-llms-openai<0.4.0,>=0.3.0, but you have llama-index-llms-openai 0.2.16 which is incompatible.
llama-index-retrievers-bm25 0.5.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-wikipedia 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-llama-parse 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-file 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-inde

#### Importing Packages

In [2]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

import sys
import shutil
import glob
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd


import llama_index

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

In [3]:
import logging

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Defining Models

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#model="gpt-3.5-turbo"
model="gpt-4o-mini"
#model="gpt-4o"

Settings.llm = OpenAI(temperature=0, model=model)
llm = Settings.llm

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
embed_model = Settings.embed_model

In [5]:
import nest_asyncio
nest_asyncio.apply()

# 1. Response Evaluation

## 1.1 Correctness  
The CorrectnessEvaluator evaluates the relevance and correctness of a generated answer against a reference answer.

In [6]:
from llama_index.core.evaluation import CorrectnessEvaluator
evaluator = CorrectnessEvaluator(llm=llm)

In [7]:
query = ("Can you explain the theory of relativity proposed by Albert Einstein in detail?")

reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, 
published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a 
vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).
General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between 
masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as 
planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated 
using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move 
towards.
In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, 
published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a 
vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) 
and mass (m).
However, general relativity, published in 1915, extended these ideas to include the effects of magnetism. According to general relativity, 
gravity is not a force between masses but rather the result of the warping of space and time by magnetic fields generated by massive objects. 
Massive objects, such as planets and stars, create magnetic fields that cause a curvature in spacetime, and smaller objects follow curved paths 
in response to this magnetic curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet with magnets 
underneath, causing it to create a depression that other objects (representing smaller masses) naturally move towards due to magnetic attraction.
"""

In [8]:
result = evaluator.evaluate(query=query, 
                            response=response, 
                            reference=reference,)
print(result.score)
print(result.feedback)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2.0
The generated answer is relevant to the user query as it discusses the theory of relativity, but it contains significant inaccuracies, particularly in its explanation of general relativity, which incorrectly attributes the warping of spacetime to magnetic fields instead of mass and energy. This misrepresentation affects the correctness of the answer.


## 1.2 Semantic Similarity  
The SemanticSimilarityEvaluator evaluates the quality of a question answering system via semantic similarity.  
Concretely, it calculates the similarity score between embeddings of the generated answer and the reference answer.  

In [9]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator
evaluator = SemanticSimilarityEvaluator()

In [10]:
# This evaluator only uses `response` and `reference`. The passing in query does not influence the evaluation
# query = 'What is the color of the sky'

response = "The sky is typically blue"

reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, 
and location. During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere 
scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter,
smaller waves. This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(response=response, 
                                   reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Score:  0.8739485436676929
Passing:  True


In [11]:
response = "Sorry, I do not have sufficient context to answer this question."

reference = """The color of the sky can vary depending on several factors, including time of day, weather conditions, 
and location. During the day, when the sun is in the sky, the sky often appears blue. 
This is because of a phenomenon called Rayleigh scattering, where molecules and particles in the Earth's atmosphere 
scatter sunlight in all directions, and blue light is scattered more than other colors because it travels as shorter,
smaller waves. This is why we perceive the sky as blue on a clear day.
"""

result = await evaluator.aevaluate(response=response, reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)  # default similarity threshold is 0.8

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Score:  0.7216314937446562
Passing:  False


### Customization of matching function and threshold:

In [13]:
from llama_index.core.embeddings import resolve_embed_model
evaluator = SemanticSimilarityEvaluator(embed_model=embed_model, 
                                        similarity_threshold=0.6,)

In [14]:
response = "The sky is yellow."
reference = "The sky is blue."

result = await evaluator.aevaluate(response=response, reference=reference,)
print("Score: ", result.score)
print("Passing: ", result.passing)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Score:  0.9405787817542284
Passing:  True


We note here that a high score does not imply the answer is always correct.   
Embedding similarity primarily captures the notion of "relevancy". 

Since both the response and reference discuss "the sky" and colors, they are semantically similar.

## 1.3 Faithfulness

The `FaithfulnessEvaluator` module measures if the response from a query engine matches any source nodes.  
This is useful for measuring if the response was hallucinated.  

In [20]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Response

from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.evaluation import EvaluationResult
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

#### The data is extracted from the [New York City](https://en.wikipedia.org/wiki/New_York_City) wikipedia page.

In [22]:
documents = SimpleDirectoryReader(input_files=["../../Data/nyc_text.txt"]).load_data()
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


#### Defining a jupyter display function

In [23]:
def display_eval_df(response: Response, eval_result: EvaluationResult) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

To run evaluations you can call the `.evaluate_response()` function on the `Response` object return from the query to run the evaluations.  
Lets evaluate the outputs of the vector_index.

In [21]:
evaluator = FaithfulnessEvaluator(llm=llm)

In [37]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query("How did New York City get its name?")
eval_result = evaluator.evaluate_response(response=response_vector)
display_eval_df(response_vector, eval_result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


,Response,Source,Evaluation Result,Reasoning
0,"New York City was named in honor of the Duke of York, who would later become King James II of England. The name was given in 1664 when England seized the territory from Dutch control, and the Duke was appointed as the proprietor of the former territory of New Netherland, which included the city of New Amsterdam.","The city came under British control in 1664 and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city was regained by the Dutch in July 1673 and was renamed New Orange for one year and three months; the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries, and is a symbol of the U.S. and its ideals of liberty and peace. In the 21st century, New York City has emerged as a global node of creativity, entrepreneurship, and as a symbol of freedom and cultural diversity. The New York Times has won the most Pulitzer Prizes for journalism and remains the U.S. media's ""newspaper of record"". In 2019, New York City was voted the greatest city in the world in a survey of over 30,000 p...",Pass,YES


#### Benchmark on Generated Questions

Now lets generate a few more questions so that we have more to evaluate with and run a small benchmark.

In [25]:
#from llama_index.core.evaluation import DatasetGenerator
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

#question_generator = DatasetGenerator.from_documents(documents)
question_generator = RagDatasetGenerator.from_documents(documents)
eval_questions = question_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/comp

In [26]:
eval_questions.to_pandas().head(2)

query  \
0  What are the five boroughs of New York City, and how are they related to the counties of New York State?        
1  Discuss the historical significance of the Statue of Liberty in relation to immigration in the United States.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [30]:
for q in [e.query for e in eval_questions.examples[0:5]]:
    print(f"- {q}")

- What are the five boroughs of New York City, and how are they related to the counties of New York State?
- Discuss the historical significance of the Statue of Liberty in relation to immigration in the United States.
- How does New York City's metropolitan economy compare to that of other global cities, and what would its gross metropolitan product rank if it were a sovereign state?
- What significant cultural and economic roles does New York City play on a global scale, as mentioned in the context information?
- Describe the early history of New York City, including the indigenous people who inhabited the area and the first documented European exploration.


In [38]:
import asyncio

def evaluate_query_engine(query_engine, questions):
    c = [query_engine.aquery(q) for q in questions]
    results = asyncio.run(asyncio.gather(*c))
    print("finished query")

    total_correct = 0
    for r in results:
        # evaluate with gpt 4
        eval_result = (1 if evaluator.evaluate_response(response=r).passing else 0)
        total_correct += eval_result

    return total_correct, len(results)

In [39]:
vector_query_engine = vector_index.as_query_engine()
correct, total = evaluate_query_engine(vector_query_engine, [e.query for e in eval_questions.examples[0:5]]) #eval_questions[:5])

print(f"score: {correct}/{total}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http

#### The response was faithful regarding the context 80% of the time.

## 1.4 Guideline Adherence  
GuidelineEvaluator evaluates a question answer system given user specified guidelines.

In [40]:
from llama_index.core.evaluation import GuidelineEvaluator
import nest_asyncio
nest_asyncio.apply()

In [41]:
GUIDELINES = ["The response should fully answer the query.",
              "The response should avoid being vague or ambiguous.",
              "The response should be specific and use statistics or numbers when possible.",
             ]

evaluators = [GuidelineEvaluator(llm=llm, guidelines=guideline) for guideline in GUIDELINES]

In [42]:
sample_data = {
    "query": "Tell me about global warming.",
    "contexts": [
        (
            "Global warming refers to the long-term increase in Earth's"
            " average surface temperature due to human activities such as the"
            " burning of fossil fuels and deforestation."
        ),
        (
            "It is a major environmental issue with consequences such as"
            " rising sea levels, extreme weather events, and disruptions to"
            " ecosystems."
        ),
        (
            "Efforts to combat global warming include reducing carbon"
            " emissions, transitioning to renewable energy sources, and"
            " promoting sustainable practices."
        ),
    ],
    "response": (
        "Global warming is a critical environmental issue caused by human"
        " activities that lead to a rise in Earth's temperature. It has"
        " various adverse effects on the planet."
    ),
}

In [43]:
for guideline, evaluator in zip(GUIDELINES, evaluators):
    eval_result = evaluator.evaluate(
        query=sample_data["query"],
        contexts=sample_data["contexts"],
        response=sample_data["response"],
    )
    print("=====")
    print(f"Guideline: {guideline}")
    print(f"Pass: {eval_result.passing}")
    print(f"Feedback: {eval_result.feedback}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=====
Guideline: The response should fully answer the query.
Pass: False
Feedback: The response provides a brief overview of global warming but lacks depth and detail. It mentions that global warming is caused by human activities and has adverse effects, but it does not specify what those activities are or what the effects might be. A more comprehensive answer would include information on greenhouse gas emissions, the role of fossil fuels, specific impacts on ecosystems and weather patterns, and potential solutions or mitigation strategies.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=====
Guideline: The response should avoid being vague or ambiguous.
Pass: False
Feedback: The response is some

## 1.5 Question Generation  

We will walk again through the process of generating a list of questions that could be asked about your data.  
This is useful for setting up an evaluation pipeline using the **FaithfulnessEvaluator** and **RelevancyEvaluator** evaluation tools.

In [48]:
#from llama_index.core.evaluation import DatasetGenerator, 
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.llama_dataset import LabelledRagDataset
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response

#import logging
#import sys
#import pandas as pd

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [49]:
reader = SimpleDirectoryReader(input_files=[f"../../Data/paul_graham_essay.txt"])
documents = reader.load_data()

In [50]:
data_generator = RagDatasetGenerator.from_documents(documents)

In [51]:
#eval_questions = data_generator.generate_questions_from_nodes()
eval_questions = LabelledRagDataset.from_json("../../Data/rag_dataset.json")

In [52]:
eval_questions.to_pandas().head(5)

query  \
0  "Describe the author's early experiences with programming. What was the first machine he used and what challenges did he face while trying to write programs on it?"                                    
1  "What were the author's initial career aspirations before college and how did his experiences in college change his perspective and career path?"                                                       
2  "Discuss the influence of the novel 'The Moon is a Harsh Mistress' and the PBS documentary featuring Terry Winograd on the author's decision to switch to AI. Why did these two sources inspire him?"   
3  "In the context, the author mentions a novel by Heinlein that influenced his interest in AI. What is the name of this novel and how did it inspire the author's interest in AI?"                        
4  "The author discusses his experience with learning Lisp and how it expanded his concept of a program. Can you explain what Lisp is and why it was considered the language of AI during that time?"      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [54]:
for q in [e.query for e in eval_questions.examples[0:5]]:
    print(f"- {q}")

- "Describe the author's early experiences with programming. What was the first machine he used and what challenges did he face while trying to write programs on it?"
- "What were the author's initial career aspirations before college and how did his experiences in college change his perspective and career path?"
- "Discuss the influence of the novel 'The Moon is a Harsh Mistress' and the PBS documentary featuring Terry Winograd on the author's decision to switch to AI. Why did these two sources inspire him?"
- "In the context, the author mentions a novel by Heinlein that influenced his interest in AI. What is the name of this novel and how did it inspire the author's interest in AI?"
- "The author discusses his experience with learning Lisp and how it expanded his concept of a program. Can you explain what Lisp is and why it was considered the language of AI during that time?"


#### Saving our Dataset

In [55]:
eval_questions.save_json("../../Data/rag_dataset.json")

#### Reading the saved Dataset

In [57]:
eval_questions = LabelledRagDataset.from_json("../../Data/rag_dataset.json")

#### Using the Dataset for Evaluation (creating an index with the same source of the questions)

In [58]:
evaluator = RelevancyEvaluator(llm=llm)

In [59]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


#### Defining another jupyter display function

In [70]:
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response.response),
            "Source": (response.source_nodes[0].node.get_content()[:1000] + "..."),
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [66]:
eval_questions[1].query

'"What were the author\'s initial career aspirations before college and how did his experiences in college change his perspective and career path?"'

In [68]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[2].query)
eval_result = evaluator.evaluate_response(query=eval_questions[1].query, response=response_vector)
display_eval_df(eval_questions[1].query, response_vector, eval_result.feedback)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


,Query,Response,Source,Evaluation Result
0,"""What were the author's initial career aspirations before college and how did his experiences in college change his perspective and career path?""","The novel ""The Moon is a Harsh Mistress"" and the PBS documentary featuring Terry Winograd had a significant impact on the author's decision to switch to AI. The novel, which features an intelligent computer named Mike, captivated the author and created a sense of anticipation about the future of AI. It painted a vivid picture of a world where intelligent machines could exist, leading him to believe that such advancements were imminent. Similarly, the PBS documentary showcasing Winograd using SHRDLU, an early natural language processing program, further fueled his enthusiasm. The demonstration suggested that the development of intelligent systems was not just a distant dream but something that could be realized in the near future. The author was inspired by the idea that with enough knowledge and effort, it was possible to teach machines to understand and process human language, which aligned with his growing interest in AI. Together, these sources instilled a sense of excitement and possibility, prompting the author to shift his focus from philosophy, which he found unfulfilling, to the burgeoning field of artificial intelligence.","I couldn't have put this into words when I was 18. All I knew at the time was that I kept taking philosophy courses and they kept being boring. So I decided to switch to AI. AI was in the air in the mid 1980s, but there were two things especially that made me want to work on it: a novel by Heinlein called The Moon is a Harsh Mistress, which featured an intelligent computer called Mike, and a PBS documentary that showed Terry Winograd using SHRDLU. I haven't tried rereading The Moon is a Harsh Mistress, so I don't know how well it has aged, but when I read it I was drawn entirely into its world. It seemed only a matter of time before we'd have Mike, and when I saw Winograd using SHRDLU, it seemed like that time would be a few years at most. All you had to do was teach SHRDLU more words. There weren't any classes in AI at Cornell then, not even graduate classes, so I started trying to teach myself. Which meant learning Lisp, since in those days Lisp was regarded as the language of ...",YES


In [69]:
print(eval_result.passing)
print(eval_result.score)
print(eval_result.feedback)

True
1.0
YES


# 2 Dataset generation  
## 2.1 Benchmarking RAG Pipelines with a `LabelledRagDatatset`

The `LabelledRagDataset` is meant to be used for evaluating any given RAG pipeline, for which there could be several configurations (i.e. choosing the `LLM`, values for the `similarity_top_k`, `chunk_size`, and others). 
This relates to traditional machine learning datasets, where `X` features are meant to predict a ground-truth label `y`. In this case, we use the `query` as well as the retrieved `contexts` as the "features" and the answer to the query, called `reference_answer` as the ground-truth label.

And of course, such datasets are comprised of observations or examples. In the case of `LabelledRagDataset`, these are made up with a set of `LabelledRagDataExample`'s.

Let's construct a `LabelledRagDataset` from scratch. Please note that the alternative to this would be to simply download a community supplied `LabelledRagDataset` from `llama-hub` in order to evaluate/benchmark your own RAG pipeline on it.

In [72]:
from llama_index.core.llama_dataset import (
    LabelledRagDataExample,
    CreatedByType,
    CreatedBy,
)

# constructing a LabelledRagDataExample
query = "This is a test query, is it not?"
query_by = CreatedBy(type=CreatedByType.AI, model_name="gpt-4")
reference_answer = "Yes it is."
reference_answer_by = CreatedBy(type=CreatedByType.HUMAN)
reference_contexts = ["This is a sample context"]

rag_example = LabelledRagDataExample(
    query=query,
    query_by=query_by,
    reference_contexts=reference_contexts,
    reference_answer=reference_answer,
    reference_answer_by=reference_answer_by,
)

The `LabelledRagDataExample` is a Pydantic `Model` and so, going from `json` or `dict` (and vice-versa) is possible.

JSON view

In [73]:
print(rag_example.json())

{"query":"This is a test query, is it not?","query_by":{"model_name":"gpt-4","type":"ai"},"reference_contexts":["This is a sample context"],"reference_answer":"Yes it is.","reference_answer_by":{"model_name":"","type":"human"}}


In [74]:
LabelledRagDataExample.parse_raw(rag_example.json())

LabelledRagDataExample(query='This is a test query, is it not?', query_by=CreatedBy(model_name='gpt-4', type=<CreatedByType.AI: 'ai'>), reference_contexts=['This is a sample context'], reference_answer='Yes it is.', reference_answer_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>))

Dictionary view

In [75]:
rag_example.dict()

{'query': 'This is a test query, is it not?',
 'query_by': {'model_name': 'gpt-4', 'type': <CreatedByType.AI: 'ai'>},
 'reference_contexts': ['This is a sample context'],
 'reference_answer': 'Yes it is.',
 'reference_answer_by': {'model_name': '',
  'type': <CreatedByType.HUMAN: 'human'>}}

In [76]:
LabelledRagDataExample.parse_obj(rag_example.dict())

LabelledRagDataExample(query='This is a test query, is it not?', query_by=CreatedBy(model_name='gpt-4', type=<CreatedByType.AI: 'ai'>), reference_contexts=['This is a sample context'], reference_answer='Yes it is.', reference_answer_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>))

#### Let's create a second example, so we can have a (slightly) more interesting `LabelledRagDataset`.

In [77]:
query = "This is a test query, is it so?"
reference_answer = "I think yes, it is."
reference_contexts = ["This is a second sample context"]

rag_example_2 = LabelledRagDataExample(
    query=query,
    query_by=query_by,
    reference_contexts=reference_contexts,
    reference_answer=reference_answer,
    reference_answer_by=reference_answer_by,
)

### The `LabelledRagDataset` Class

In [78]:
from llama_index.core.llama_dataset import LabelledRagDataset

new_rag_dataset = LabelledRagDataset(examples=[rag_example, rag_example_2])
new_rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"This is a test query, is it not?",[This is a sample context],Yes it is.,human,ai (gpt-4)
1,"This is a test query, is it so?",[This is a second sample context],"I think yes, it is.",human,ai (gpt-4)


To persist and load the dataset to and from disk, there are the `save_json` and `from_json` methods.

In [79]:
new_rag_dataset.save_json("../../Data/new_rag_dataset.json")
reload_rag_dataset = LabelledRagDataset.from_json("../../Data/rag_dataset.json")

#### Building a synthetic `LabelledRagDataset` over Wikipedia 

For this section, we'll first create a `LabelledRagDataset` using a synthetic generator. Ultimately, we will use GPT-4 to produce both the `query` and `reference_answer` for the synthetic `LabelledRagDataExample`'s.

NOTE: if one has queries, reference answers, and contexts over a text corpus, then it is not necessary to use data synthesis to be able to predict and subsequently evaluate said predictions.

In [80]:
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
nest_asyncio.apply()

In [81]:
#%pip install -q llama-index-llms-openai
%pip install -qU llama-index-readers-wikipedia
%pip install -q wikipedia

In [85]:
#from llama_index.llms.openai import OpenAI
#from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import VectorStoreIndex

#### Creating a small Corpus from wikipedia

In [86]:
cities = ["Vienna",]
documents = WikipediaReader().load_data(pages=[f"History of {x}" for x in cities])
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


The `RagDatasetGenerator` can be built over a set of documents to generate `LabelledRagDataExample`'s.

#### Generating questions against chunks  
#### Instantiating a DatasetGenerator

In [87]:
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=2,  # set the number of questions per nodes
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [88]:
len(dataset_generator.nodes)

8

In [89]:
# since there are 8 nodes, there should be a total of 16 questions
rag_dataset = dataset_generator.generate_dataset_from_nodes()

  0%|          | 0/8 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 12%|█▎        | 1/8 [00:01<00:08,  1.25s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 25%|██▌       | 2/8 [00:01<00:04,  1.27it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 50%|█████     | 1/2 [00:07<00:07,  7.21s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 2/2 [00:12<00:00,  6.20s/it]


In [90]:
rag_dataset.to_pandas().head(5)

query  \
0  Discuss the significance of the year 1155 in the history of Vienna and explain how it contributed to the city's development under the Babenberg dynasty.                                                                  
1  What were the implications of Vienna receiving the rights of a city and staple port in 1221, and how did this status affect its role in trade within the Holy Roman Empire?                                               
2  Discuss the significance of Rudolf IV of Austria's contributions to Vienna during the Habsburg rule, particularly in relation to the establishment of the University of Vienna and the construction of the Stephansdom.   
3  Analyze the impact of the First Turkish Siege of 1529 on Vienna's fortifications and military strategy, and explain how these developments influenced the outcome of the Second Turkish Siege in 1683.                    
4  Discuss the impact of Emperor Joseph II's administration reforms in 1783 on the city of Vienna, particularly in relation to urban management and public health.                                                           

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [91]:
rag_dataset.save_json("../../Data/Vienna_dataset.json")

### 1.7 Context Relevancy and Answer Relevancy  
`AnswerRelevancyEvaluator` and `ContextRelevancyEvaluator` give a measure on the relevancy of a generated answer and retrieved contexts, respectively, to a given user query. Both of these evaluators return a `score` that is between 0 and 1 as well as a generated `feedback` explaining the score. Note that, higher score means higher relevancy. In particular, we prompt the judge LLM to take a step-by-step approach in providing a relevancy score, asking it to answer the following two questions of a generated answer to a query for answer relevancy (for context relevancy these are slightly adjusted):

1. Does the provided response match the subject matter of the user's query?
2. Does the provided response attempt to address the focus or perspective on the subject matter taken on by the user's query?

Each question is worth 1 point and so a perfect evaluation would yield a score of 2/2.

First, we build a RAG over the same source documents used to created the `rag_dataset`.

In [92]:
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


With our RAG (i.e `query_engine`) defined, we can make predictions (i.e., generate responses to the query) with it over the `rag_dataset`.

In [93]:
prediction_dataset = await rag_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=100, show_progress=True
)

Batch processing of predictions:   0%|          | 0/16 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

Batch processing of predictions:   6%|▋         | 1/16 [00:04<01:08,  4.56s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batch processing of predictions:  12%|█▎        | 2/16 [00:04<00:28,  2.03s/it]

...

Batch processing of predictions:  88%|████████▊ | 14/16 [00:08<00:00,  3.16it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batch processing of predictions:  94%|█████████▍| 15/16 [00:09<00:00,  1.96it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batch processing of predictions: 100%|██████████| 16/16 [00:10<00:00,  1.58it/s]


### Evaluating Answer and Context Relevancy Separately

We first need to define our evaluators (i.e. `AnswerRelevancyEvaluator` & `ContextRelevancyEvaluator`):

In [94]:
# instantiate the gpt-4 judges
#from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    AnswerRelevancyEvaluator,
    ContextRelevancyEvaluator,
)

judges = {}

judges["answer_relevancy"] = AnswerRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-3.5-turbo"),
)

judges["context_relevancy"] = ContextRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-4"),
)

Now, we can use our evaluator to make evaluations by looping through all of the <example, prediction> pairs.

In [95]:
eval_tasks = []
for example, prediction in zip(
    rag_dataset.examples, prediction_dataset.predictions
):
    eval_tasks.append(
        judges["answer_relevancy"].aevaluate(
            query=example.query,
            response=prediction.response,
            sleep_time_in_seconds=10.0,
        )
    )
    eval_tasks.append(
        judges["context_relevancy"].aevaluate(
            query=example.query,
            contexts=prediction.contexts,
            sleep_time_in_seconds=10.0,
        )
    )

In [96]:
eval_results1 = await tqdm_asyncio.gather(*eval_tasks[:8])
eval_results2 = await tqdm_asyncio.gather(*eval_tasks[8:16])
eval_results3 = await tqdm_asyncio.gather(*eval_tasks[16:24])
eval_results4 = await tqdm_asyncio.gather(*eval_tasks[24:])
eval_results = eval_results1 + eval_results2+ eval_results3 + eval_results4

  0%|          | 0/8 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 12%|█▎        | 1/8 [00:11<01:20, 11.48s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 25%|██▌       | 2/8 [00:12<00:32,  5.39s/it]

...

 75%|███████▌  | 6/8 [00:18<00:05,  2.55s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 3.054000 seconds
Retrying request to /chat/completions in 3.054000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 2.982000 seconds
Retrying request to /chat/completions in 2.982000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.7888028408087993 seconds as it raised RateLimitError: Erro

 88%|████████▊ | 7/8 [00:29<00:05,  5.28s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 1.548000 seconds
Retrying request to /chat/completions in 1.548000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 8/8 [00:44<00:00,  5.58s/it]


In [97]:
evals = {
    "answer_relevancy": eval_results[::2],
    "context_relevancy": eval_results[1::2],
}

### Taking a look at the evaluation results

Here we use a utility function to convert the list of `EvaluationResult` objects into something more notebook friendly. This utility will provide two DataFrames, one deep one containing all of the evaluation results, and another one which aggregates via taking the mean of all the scores, per evaluation method.

In [98]:
from llama_index.core.evaluation.notebook_utils import get_eval_results_df
import pandas as pd

deep_dfs = {}
mean_dfs = {}
for metric in evals.keys():
    deep_df, mean_df = get_eval_results_df(
        names=["baseline"] * len(evals[metric]),
        results_arr=evals[metric],
        metric=metric,
    )
    deep_dfs[metric] = deep_df
    mean_dfs[metric] = mean_df

In [99]:
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,1.000000
mean_context_relevancy_score,0.507812


The above utility also provides the mean score across all of the evaluations in `mean_df`.

We can get a look at the raw distribution of the scores by invoking `value_counts()` on the `deep_df`.

In [100]:
deep_dfs["answer_relevancy"]["scores"].value_counts()

scores
1.0    16
Name: count, dtype: int64

In [101]:
deep_dfs["context_relevancy"]["scores"].value_counts()

scores
0.750    4
0.250    3
0.625    2
0.375    2
0.000    2
1.000    1
0.875    1
0.500    1
Name: count, dtype: int64

It looks like for the most part, the default RAG does fairly well in terms of generating answers that are relevant to the query. Getting a closer look is made possible by viewing the records of any of the `deep_df`'s.

In [102]:
deep_dfs["context_relevancy"].head(2)

rag  \
0  baseline   
1  baseline   

                                                                                                                                                                         query  \
0  Discuss the significance of the year 1155 in the history of Vienna and explain how it contributed to the city's development under the Babenberg dynasty.                      
1  What were the implications of Vienna receiving the rights of a city and staple port in 1221, and how did this status affect its role in trade within the Holy Roman Empire?   

  answer  \
0  None    
1  None    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

And, of course you can apply any filters as you like. For example, if you want to look at the examples that yielded less than perfect results.

In [103]:
cond = deep_dfs["context_relevancy"]["scores"] < 0.6
deep_dfs["context_relevancy"][cond].head(5)

rag  \
3   baseline   
4   baseline   
5   baseline   
11  baseline   
12  baseline   

                                                                                                                                                                                                                         query  \
3   Analyze the impact of the First Turkish Siege of 1529 on Vienna's fortifications and military strategy, and explain how these developments influenced the outcome of the Second Turkish Siege in 1683.                       
4   Discuss the impact of Emperor Joseph II's administration reforms in 1783 on the city of Vienna, particularly in relation to urban management and public health.                                                              
5   Analyze the significance of the Congress of Vienna (1814-1815) in reshaping the political landscape of Europe, and explain how it affected Austria's financial situation.                                                    
11  Explain the significance of the Austrian State Treaty of 1955 and its implications for Austria's political status and neutrality in the context of post-war Europe.                                                          
12  Discuss the significance of Karl Lueger's mayoralty in fin-de-siècle Vienna as presented in Richard S. Geehr's work. How did his leadership influence the political and cultural landscape of the city during that period?   

   answer  \
3   None    
4   None    
5   None    
11  None    
12  None    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## 2.2 Retrieval Evaluation  

Given a retriever and a set of questions, evaluate retrieved results using ranking metrics.

https://docs.llamaindex.ai/en/stable/module_guides/evaluating/
https://docs.llamaindex.ai/en/stable/module_guides/evaluating/usage_pattern_retrieval/
https://docs.llamaindex.ai/en/stable/module_guides/evaluating/usage_pattern/